In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

In [2]:
df = pd.read_csv('Dataset/dataset.csv')
df.head()

,X,Y,Z,Mixed,ClassLabel
0,0.125022,0.094986,0.001297,0.157018,1
1,0.150710,0.083282,-0.023514,0.173788,1
2,0.102941,0.111084,0.010075,0.151782,1
3,0.038450,0.049911,0.007511,0.063451,1
4,-0.029148,-0.105423,0.017124,0.110711,1


In [3]:
person1 = df[df['ClassLabel'] == 1]
person2 = df[df['ClassLabel'] == 2]
person3 = df[df['ClassLabel'] == 2]
person4 = df[df['ClassLabel'] == 4]
person5 = df[df['ClassLabel'] == 5]

In [5]:
# Define functions for outlier removal and metric calculation
def remove_outliers_iqr(data, column):
    q1 = data[column].quantile(0.25)
    q3 = data[column].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    return data[(data[column] >= lower_bound) & (data[column] <= upper_bound)]

In [6]:
def calculate_metrics(window):
    mean = window.mean()
    std_dev = window.std()
    energy = np.sum(np.square(window))
    hist = np.histogram(window, bins=10, density=True)[0]
    entropy = -np.sum(hist * np.log2(hist + 1e-10))
    peaks, _ = find_peaks(window, height=0)
    num_peaks = len(peaks)
    return mean, std_dev, energy, entropy, num_peaks

In [8]:
window_size = 100
all_metrics = []
personas = [person1, person2, person3, person4, person5]

In [10]:
window_size = 100
all_metrics = []
personas = [person1, person2, person3, person4, person5]

for index, person_df in enumerate(personas):
    # Remove outliers for each column
    for col in ['X', 'Y', 'Z', 'Mixed']:
        person_df = remove_outliers_iqr(person_df, col)

    # Calculate metrics for each column
    metrics = {}
    for col in ['X', 'Y', 'Mixed']:
        for metric_name in ['Mean', 'Std Dev', 'Energy', 'Entropy', 'Peaks']:
            metrics[f'{metric_name}_{col}'] = []

    for i in range(0, len(person_df)):
        for col in ['X', 'Y', 'Mixed']:
            window = person_df[col].iloc[i:i + window_size]
            mean, std_dev, energy, entropy, num_peaks = calculate_metrics(window)
            metrics[f'Mean_{col}'].append(mean)
            metrics[f'Std Dev_{col}'].append(std_dev)
            metrics[f'Energy_{col}'].append(energy)
            metrics[f'Entropy_{col}'].append(entropy)
            metrics[f'Peaks_{col}'].append(num_peaks)

    result_df = pd.DataFrame(metrics)
    result_df['category'] = index + 1  # Add category based on index (+1 to start from 1)
    all_metrics.append(result_df)

# Concatenate all DataFrames into one
combined_df = pd.concat(all_metrics)

# Save combined DataFrame to CSV
combined_df.to_csv('combined_metrics.csv', index=False)


In [11]:
combined_df.describe()

,Mean_X,Std Dev_X,Energy_X,Entropy_X,Peaks_X,Mean_Y,Std Dev_Y,Energy_Y,Entropy_Y,Peaks_Y,Mean_Mixed,Std Dev_Mixed,Energy_Mixed,Entropy_Mixed,Peaks_Mixed,category
count,4388.000000,4383.000000,4388.000000,4388.000000,4388.000000,4388.000000,4383.000000,4388.000000,4388.000000,4388.000000,4388.000000,4383.000000,4388.000000,4388.000000,4388.000000,4388.000000
mean,0.004006,0.089387,1.072148,-158.810492,15.977666,0.009437,0.135307,2.314744,-252.328564,19.127165,0.160095,0.080044,4.065248,-314.594699,27.043072,2.997949
std,0.024133,0.056847,1.001885,764.138718,4.292311,0.024470,0.076748,2.099678,9279.406230,5.218609,0.096438,0.046162,3.680242,7930.806030,5.782330,1.412841
min,-0.191591,0.002174,0.000001,-34694.374782,0.000000,-0.142838,0.000227,0.000059,-434709.340039,0.000000,0.008352,0.000261,0.000073,-371539.304425,0.000000,1.000000
25%,-0.002825,0.024238,0.058274,-300.552498,14.000000,-0.004412,0.051360,0.261709,-102.262692,17.000000,0.051069,0.028717,0.341078,-285.032911,27.000000,2.000000
50%,0.000338,0.103842,1.025896,-27.489962,17.000000,0.003829,0.142736,1.926557,-13.497040,20.000000,0.179642,0.085308,3.665701,-47.255963,28.000000,3.000000
75%,0.011288,0.139175,1.921716,-14.100193,19.000000,0.019464,0.206085,4.195303,-3.364960,23.000000,0.252299,0.124242,7.467996,-20.995007,30.000000,4.000000
max,0.258800,0.238033,3.431192,-6.108219,24.000000,0.257938,0.316833,7.932150,0.710296,30.000000,0.358251,0.170074,12.525864,-12.686242,35.000000,5.000000


In [12]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4388 entries, 0 to 846
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Mean_X         4388 non-null   float64
 1   Std Dev_X      4383 non-null   float64
 2   Energy_X       4388 non-null   float64
 3   Entropy_X      4388 non-null   float64
 4   Peaks_X        4388 non-null   int64  
 5   Mean_Y         4388 non-null   float64
 6   Std Dev_Y      4383 non-null   float64
 7   Energy_Y       4388 non-null   float64
 8   Entropy_Y      4388 non-null   float64
 9   Peaks_Y        4388 non-null   int64  
 10  Mean_Mixed     4388 non-null   float64
 11  Std Dev_Mixed  4383 non-null   float64
 12  Energy_Mixed   4388 non-null   float64
 13  Entropy_Mixed  4388 non-null   float64
 14  Peaks_Mixed    4388 non-null   int64  
 15  category       4388 non-null   int64  
dtypes: float64(12), int64(4)
memory usage: 582.8 KB
